<h1 style="background-color:#1c6ce6;
           font-family:segoe ui;
                color:#FFF9ED;
           font-size:300%;
           text-align:center;
           border-radius:10px 10px;"> 
✍🏽 תהליך הסבת שכבת גבולות תכניות לצרכי רישום לבנק"ל מודרני ✍🏽 

In [ ]:
print(' Tazar borders:')

<h1 style="text-align:right; font-size:150%; color:#1c6ce6;">
ספריות 📚

In [ ]:
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
import polars as pl
import datetime as dt
import geopandas as gpd
from configs import CNFG
import dask_geopandas as daskgpd
from helpers import assign_ProcessName, assign_BlockName, drop_bad_ProcessNames

<h1 style="text-align:right; font-size:150%; color:#1c6ce6;">קריאת נתונים 📑 

In [ ]:
print('   Reading inputs')

<h1 style="text-align:right;font-size:100%">
טבלת תצ"ר עם קידומת

In [ ]:
tazar_kidomet_cols = ['TALAR_NUM', 'TALAR_YEAR', 'KIDOMET_TALAR']

tazar_kidomet = pd.read_csv(CNFG.Local + 'KIDOMET_TALAR.csv', usecols = tazar_kidomet_cols, encoding = 'cp1255').astype({'TALAR_NUM':int, 'TALAR_YEAR':int})

del tazar_kidomet_cols

<h1 style="text-align:right;font-size:100%">
טבלת תצ"ר בתהליך

In [ ]:
general_inProcess_cols = {'columns' : ['TALAR_NUM', 'TALAR_YEAR', 'CADASTER_PROCESS', 'GUSHNUM', 'GUSHSUFFIX', 'SURVEYOR', 'TABA_NAMES']}

general_inProcess_schema = {'TALAR_NUM' : 'Int64', 'TALAR_YEAR' : 'Int64', 'CADASTER_PROCESS' : 'Int64', 'SURVEYOR' : 'Int64',
                            'GUSHNUM'   : 'Int64', 'GUSHSUFFIX' : 'Int64', 'TABA_NAMES'       : str}

general_inProcess = pl.read_excel(CNFG.Oracle + 'general_inProcess.xlsx', read_csv_options = general_inProcess_cols)\
                      .to_pandas()\
                      .astype(general_inProcess_schema)\
                      .rename(columns = {'GUSHNUM' : 'GUSH_NUM', 'GUSHSUFFIX' : 'GUSH_SUFFIX', 'TABA_NAMES' : 'PlanName'})

del [general_inProcess_cols, general_inProcess_schema]

<h1 style="text-align:right;font-size:100%">
שכבת גבולות תצ"ר

In [ ]:
tazar_border_cols = ['GUSH_NUM', 'GUSH_SUFFIX', 'SURVEYOR', 'STATUS_TEXT', 'STATUS', 'TALAR_SOURCE', 'TALAR_YEAR', 'TALAR_NUM', 'TALAR_ID', 'geometry']

tazar_border_schema = {'GUSH_NUM' : int, 'GUSH_SUFFIX' : int, 'SURVEYOR' : int, 'STATUS_TEXT': str,
                       'STATUS' : int, 'TALAR_SOURCE': int, 'TALAR_YEAR' : int, 'TALAR_NUM' : int, 'TALAR_ID' : int}

tazar_border = daskgpd.read_file(CNFG.CADSDE + 'TALAR.gdb', layer = 'TALAR', columns = tazar_border_cols, npartitions = 5).compute()
tazar_border = tazar_border[~tazar_border['TALAR_NUM'].isna()]


tazar_border.fillna({col:0 for col in ['GUSH_NUM','GUSH_SUFFIX','SURVEYOR','STATUS','TALAR_SOURCE','TALAR_YEAR','TALAR_NUM','TALAR_ID']},
                    inplace = True)

tazar_border = tazar_border.astype(tazar_border_schema)


del [tazar_border_cols, tazar_border_schema]

<h1 style="text-align:right;font-size:100%">
שכבת גבולות תצ"ר טבלאיות

In [ ]:
tazar_tabular_border_cols = ['SOURCE', 'STATUS_TEXT', 'STATUS', 'TALAR_YEAR', 'TALAR_NUM', 'TALAR_ID', 'GUSH_SUFFIX', 'GUSH_NUM', 'geometry']

tazar_tabular_border = daskgpd.read_file(CNFG.CADSDE + 'TALAR_TABLE.gdb', layer = 'TALAR_TABLE',  columns = tazar_tabular_border_cols, npartitions = 5).compute()

del [tazar_tabular_border_cols]

<h1 style="text-align:right;font-size:100%">
שכבת גבולות תצ"ר בתהליך

In [ ]:
tazar_inprocess_cols = ['GUSHSUFFIX', 'GUSHNUM', 'TALAR_YEAR', 'TALAR_NUM', 'CADASTER_PROCESS', 'geometry']

tazar_inprocess = daskgpd.read_file(CNFG.CADSDE + 'CadasterProcessBorder.gdb', layer = 'CadasterProcessBorder', columns = tazar_inprocess_cols, npartitions = 5).compute()

tazar_inprocess.rename(columns = {'GUSHNUM':'GUSH_NUM', 'GUSHSUFFIX':'GUSH_SUFFIX'}, inplace=True)

tazar_inprocess = tazar_inprocess[tazar_inprocess['CADASTER_PROCESS'] == 2]
tazar_inprocess.drop(columns = ['CADASTER_PROCESS'], inplace = True)


del tazar_inprocess_cols

<h1 style="text-align:right;font-size:100%">
טבלת תצ"ר טבלאי

In [ ]:
tazar_table_cols = ['GUSH_NUM', 'GUSH_SUFFIX', 'TALAR_ID', 'TALAR_NUM', 'TALAR_YEAR', 'STATUS', 'SOURCE']

tazar_table = pd.read_csv(CNFG.Oracle + 'TALAR_TABLE.csv', usecols = tazar_table_cols, encoding = 'cp1255')

del tazar_table_cols

<h1 style="text-align:right;font-size:100%">
    טבלת תצ"ר

In [ ]:
oracle_talar_cols = ['TALAR_ID', 'TALAR_NUM', 'TALAR_YEAR', 'GUSH_NUM', 'GUSH_SUFFIX', 'SURVEY_DATE', 'TALAR_SOURCE', 'STATUS', 'SURVEYOR']

oracle_talar =  pd.read_csv(CNFG.Oracle + 'TALAR.csv', usecols = oracle_talar_cols, encoding='cp1256')

del oracle_talar_cols

<h1 style="text-align:right;font-size:100%">
טבלת מערכת ניהול קדסטר

In [ ]:
hagi_table_cols = ['SURVEY_DATE', 'SURVEYOR', 'GUSH_NUM', 'GUSH_SUFFIX', 'STATUS', 'ProcessName']

hagi_table = pd.read_csv(CNFG.Local + 'hagi_Tazar.csv', usecols = hagi_table_cols, parse_dates = ['SURVEY_DATE'])

del hagi_table_cols

<h1 style="text-align:right;font-size:100%">
שכבה מעובדת של גבולות תצ"ר מארקסטורם

In [ ]:
arcstorm_tazars_cols = ['TALAR_NUM', 'TALAR_YEAR', 'GUSH_NUM', 'GUSH_SUFFIX', 'TALAR_SOURCE', 'STATUS', 'SURVEYOR','SURVEY_DATE', 'geometry']

arcstorm_tazars = daskgpd.read_file(CNFG.Arcstorm +'TAZAR\TAZAR.gdb', layer = 'TAZAR', columns = arcstorm_tazars_cols, npartitions = 5).to_crs(2039).compute()

del arcstorm_tazars_cols

<h1 style="text-align:right;font-size:100%">
גבולות תצ"ר בשטח לא מוסדר מודרניים

In [ ]:
unreg_tazar_layer = gpd.read_file(CNFG.Results, layer = 'PT11', include_fields = ['ProcessName'], ignore_geometry = True)

<h1 style="text-align:right;font-size:100%">
גבולות גושי קדסטר וגושי שומה

In [ ]:
Blocks = pd.read_csv(CNFG.CSVPath + 'BlockUniqueIDs.csv', usecols = ['Name', 'GlobalID', 'IsTax'])

<h1 style="text-align:right; font-size:150%; color:#1c6ce6;">
עיבוד נתונים 🛠️

In [ ]:
print('   Cleaning data')

<h1 style="text-align:right; font-size:100%">
סינון תהליכים שאינם תצ"ר מטבלת תהליכים כללית כאשר סוג תהליך של תצ"ר הוא 2

In [ ]:
general_inProcess = general_inProcess[general_inProcess['CADASTER_PROCESS'] == 2].drop(columns = 'CADASTER_PROCESS')

<h1 style="text-align:right; font-size:100%">
הסרת מספרי אלפיסה שאינם תקינים או כפולים בכל מקורות המידע

In [ ]:
tazar_kidomet        = drop_bad_ProcessNames(tazar_kidomet)
general_inProcess    = drop_bad_ProcessNames(general_inProcess)
tazar_border         = drop_bad_ProcessNames(tazar_border)
tazar_tabular_border = drop_bad_ProcessNames(tazar_tabular_border)
tazar_inprocess      = drop_bad_ProcessNames(tazar_inprocess)
tazar_table          = drop_bad_ProcessNames(tazar_table)
oracle_talar         = drop_bad_ProcessNames(oracle_talar)
arcstorm_tazars      = drop_bad_ProcessNames(arcstorm_tazars)
hagi_table           = hagi_table[(hagi_table['ProcessName'] != '0') | (hagi_table['ProcessName'].notna())].drop_duplicates(subset = ['ProcessName'])

<h1 style="text-align:right; font-size:100%">
הסרת רשומות ללא מידע של מספר גוש עבור טבלת תצ"ר

In [ ]:
tazar_table = tazar_table[tazar_table['GUSH_NUM'].notna()]

<h1 style="text-align:right; font-size:100%">
הסרת רשומות של תצ"ר שמגיע מכרטסת בטבלת תצ"ר

In [ ]:
tazar_table = tazar_table[tazar_table['SOURCE'] != 5]

<h1 style="text-align:right; font-size:100%">
תיקון ערכי תאריך

In [ ]:
date_mapping = {'/0012' : '/2012', '/0013' : '/2013',
                '/0014' : '/2014', '/0015' : '/2015',
                '/0016' : '/2016', '/0017' : '/2017',
                '/0018' : '/2018', '/0019' : '/2019',
                '/0020' : '/2020', '/0021' : '/2021',
                '/0022' : '/2022', '/0023' : '/2023'}

oracle_talar['SURVEY_DATE'] = oracle_talar['SURVEY_DATE'].replace(date_mapping, regex=True)
oracle_talar['SURVEY_DATE'] = pd.to_datetime(oracle_talar['SURVEY_DATE'], dayfirst=True)

arcstorm_tazars['SURVEY_DATE'] = pd.to_datetime(arcstorm_tazars['SURVEY_DATE'])

del date_mapping

<h1 style="text-align:right; font-size:100%">
טיפול בכפילויות של מספר מזהה עבור גבולות תצ"ר
<h1 style="text-align:right; font-size:100%">
טיפול בכפילויות של מספרי אליפסה מגבולות תצ"ר טבלאי (מבין הכפולים ישארו רשומות העונות למקור נתונים = 4)

In [ ]:
tazar_border.drop_duplicates(subset=['TALAR_ID'], keep='first', inplace = True)

In [ ]:
tazar_tabular_border = tazar_tabular_border.sort_values(['TALAR_NUM', 'TALAR_YEAR', 'SOURCE'])

tazar_tabular_border = tazar_tabular_border.drop_duplicates(['TALAR_NUM', 'TALAR_YEAR'], keep = 'first')

<h1 style="text-align:right; font-size:125%">
:הגדרת שדות של שם מפה ושם גוש מלא לכל מקור מידע

In [ ]:
print('   Data conversion')

<h1 style="text-align:right; font-size:100%">
תצ"ר קידומת

In [ ]:
tazar_kidomet['KIDOMET_TALAR'] = tazar_kidomet['KIDOMET_TALAR'].replace('', 'X')

tazar_kidomet = assign_ProcessName(tazar_kidomet, 'TALAR_NUM', 'TALAR_YEAR')
tazar_kidomet['ProcessName'] = tazar_kidomet['ProcessName'] + '/' + tazar_kidomet['KIDOMET_TALAR']
tazar_kidomet.drop(columns = 'KIDOMET_TALAR', inplace = True)

<h1 style="text-align:right; font-size:100%">
טבלת תצ"ר בתהליך

In [ ]:
general_inProcess = assign_ProcessName(general_inProcess, 'TALAR_NUM', 'TALAR_YEAR')    
general_inProcess = assign_BlockName(general_inProcess, 'GUSH_NUM', 'GUSH_SUFFIX')

<h1 style="text-align:right; font-size:100%">
גבולות תצ"ר

In [ ]:
tazar_border = assign_ProcessName(tazar_border, 'TALAR_NUM', 'TALAR_YEAR')    
tazar_border = assign_BlockName(tazar_border, 'GUSH_NUM', 'GUSH_SUFFIX')

<h1 style="text-align:right; font-size:100%">
גבולות תצ"ר טבלאי

In [ ]:
tazar_tabular_border = assign_ProcessName(tazar_tabular_border, 'TALAR_NUM', 'TALAR_YEAR')    
tazar_tabular_border = assign_BlockName(tazar_tabular_border, 'GUSH_NUM', 'GUSH_SUFFIX')

<h1 style="text-align:right; font-size:100%">
גבולות תצ"ר בתהליך

In [ ]:
tazar_inprocess = assign_ProcessName(tazar_inprocess, 'TALAR_NUM', 'TALAR_YEAR')    
tazar_inprocess = assign_BlockName(tazar_inprocess, 'GUSH_NUM', 'GUSH_SUFFIX')
tazar_inprocess = tazar_inprocess.set_geometry('geometry', crs = 2039)

<h1 style="text-align:right; font-size:100%">
טבלת תצ"ר טבלאי

In [ ]:
tazar_table = assign_ProcessName(tazar_table, 'TALAR_NUM', 'TALAR_YEAR')    
tazar_table = assign_BlockName(tazar_table, 'GUSH_NUM', 'GUSH_SUFFIX')

<h1 style="text-align:right; font-size:100%">
טבלת תצ"ר

In [ ]:
oracle_talar = assign_ProcessName(oracle_talar, 'TALAR_NUM', 'TALAR_YEAR')    
oracle_talar = assign_BlockName(oracle_talar, 'GUSH_NUM', 'GUSH_SUFFIX')

<h1 style="text-align:right; font-size:100%">
טבלת תצ"ר מערכת ניהול

In [ ]:
hagi_table = assign_BlockName(hagi_table, 'GUSH_NUM', 'GUSH_SUFFIX')

<h1 style="text-align:right; font-size:100%">
גבולות תצ"ר מארקסטורם

In [ ]:
arcstorm_tazars = assign_ProcessName(arcstorm_tazars, 'TALAR_NUM', 'TALAR_YEAR')    
arcstorm_tazars = assign_BlockName(arcstorm_tazars, 'GUSH_NUM', 'GUSH_SUFFIX')

<h1 style="text-align:right; font-size:100%">
האיחוד יוצר שכבה אחת גדולה של כל רשומות הטבלאות והשכבות

In [ ]:
mergeTalar = pd.merge(left = tazar_border, right = oracle_talar, on = 'ProcessName', how = 'outer', suffixes = ('_SDE', '_Oracle'))

mergeTalar['BlockName']  = mergeTalar['BlockName_Oracle'].fillna(mergeTalar['BlockName_SDE'])
mergeTalar['DataSource'] = mergeTalar['TALAR_SOURCE_Oracle'].fillna(mergeTalar['TALAR_SOURCE_SDE'])
mergeTalar['Status']     = mergeTalar['STATUS_Oracle'].fillna(mergeTalar['STATUS_SDE'])
mergeTalar['SurveyorID'] = mergeTalar['SURVEYOR_Oracle'].fillna(mergeTalar['SURVEYOR_SDE'])
mergeTalar['SurveyDate'] = mergeTalar['SURVEY_DATE']
mergeTalar['ORACLE_ID']  = mergeTalar['TALAR_ID_Oracle'].fillna(mergeTalar['TALAR_ID_SDE'])

mergeTalar = mergeTalar[['ProcessName', 'ORACLE_ID', 'BlockName', 'DataSource', 'Status', 'SurveyorID', 'SurveyDate', 'geometry']].set_geometry('geometry', crs = 2039)



mergeTabular = pd.merge(left = tazar_tabular_border, right = tazar_table, on = 'ProcessName', how = 'outer', suffixes = ('_SDE', '_Oracle'))

mergeTabular['BlockName']  = mergeTabular['BlockName_Oracle'].fillna(mergeTabular['BlockName_SDE'])
mergeTabular['DataSource'] = mergeTabular['SOURCE_Oracle'].fillna(mergeTabular['SOURCE_SDE'])
mergeTabular['Status']     = mergeTabular['STATUS_Oracle'].fillna(mergeTabular['STATUS_SDE'])
mergeTabular['ORACLE_ID']  = mergeTabular['TALAR_ID_Oracle'].fillna(mergeTabular['TALAR_ID_SDE'])

mergeTabular = mergeTabular[['ProcessName', 'ORACLE_ID', 'BlockName', 'DataSource', 'Status', 'geometry']].set_geometry('geometry', crs = 2039)


merge = pd.merge(left = mergeTalar, right = mergeTabular,      on = 'ProcessName', how = 'outer', suffixes = ('_Borders', '_Tabular')).set_geometry('geometry_Borders', crs = 2039)\
          .merge(                   right = tazar_inprocess,   on = 'ProcessName', how = 'outer', suffixes = ('_Talar', '_SDE.InProcess'))\
          .merge(                   right = general_inProcess, on = 'ProcessName', how = 'outer', suffixes = ('', '_Oracle.InProcess'))\
          .merge(                   right = hagi_table,        on = 'ProcessName', how = 'outer', suffixes = ('', '_Hagi'))\
          .merge(                   right = arcstorm_tazars,   on = 'ProcessName', how = 'outer', suffixes = ('', '_Arcstorm'))\
          .merge(                   right = tazar_kidomet,     on = 'ProcessName', how = 'outer')

In [ ]:
columns_by_order = sorted(merge.columns.to_list())
merge = merge[columns_by_order].reset_index(drop = True)

merge.rename(columns = {'BlockName':'BlockName_InProc', 'STATUS': 'Status_Hagi',
                        'SURVEYOR':'SurveyorID_Oracle.InProcess' ,'SURVEY_DATE':'SurveyDate_Hagi',
                        'geometry':'geometry_InProcess' ,'TALAR_SOURCE' : 'DataSource_Arcstorm'},
             inplace = True)

del [columns_by_order, mergeTalar, mergeTabular, tazar_border, general_inProcess, tazar_kidomet, tazar_tabular_border, tazar_inprocess, tazar_table, oracle_talar, arcstorm_tazars]

In [ ]:
merge = merge[~merge['ProcessName'].isin(unreg_tazar_layer['ProcessName'])]

del unreg_tazar_layer

In [ ]:
merge['Status'] = merge['Status_Tabular'].fillna(merge['Status_Borders'])\
                                         .fillna(merge['STATUS_Arcstorm'])\
                                         .fillna(merge['Status_Hagi'])\
                                         .fillna(1)

statusDict = {0 : 1, #לא ידוע - קליטת קבצים
              1 : 1, #קליטת קבצים - קליטת קבצים
              2 : 2, #בביקורת - בביקורת
              3 : 4, #כשרה לרישום - כשרה לרישום
              4 : 5, #רשומה - רשומה
              5 : 9, #תצ"ר מבוטל - מבטל
              6 : 3, # בתיקונים - הוחזר למודד
              None : 1} #ללא ערך - קליטת קבצים

merge['Status'] = merge['Status'].map(statusDict)

merge = merge.drop(columns = ['Status_Tabular', 'Status_Borders', 'STATUS_Arcstorm', 'Status_Hagi'])
del statusDict

In [ ]:
merge['BlockName'] = merge['BlockName_Borders'].fillna(merge['BlockName_Tabular'])\
                                               .fillna(merge['BlockName_Oracle.InProcess'])\
                                               .fillna(merge['BlockName_InProc'])\
                                               .fillna(merge['BlockName_Arcstorm'])\
                                               .fillna(merge['BlockName_Hagi'])

merge = merge.drop(columns = ['BlockName_Borders', 'BlockName_Tabular', 'BlockName_Oracle.InProcess', 'BlockName_InProc', 'BlockName_Arcstorm', 'BlockName_Hagi'])

In [ ]:
merge['SurveyDate'] = merge['SurveyDate'].fillna(merge['SURVEY_DATE_Arcstorm']).fillna(merge['SurveyDate_Hagi'])
merge = merge.drop(columns = ['SURVEY_DATE_Arcstorm', 'SurveyDate_Hagi'])

In [ ]:
merge['DataSource'] = merge['DataSource_Borders'].fillna(merge['DataSource_Tabular']).fillna(merge['DataSource_Arcstorm'])

digi_cond = merge['DataSource'].isin([0, 1, None]) # לא ידוע, גרפי, ללא ערך

# תכניות אנליטיות בשנת או מעל שנת 2015
CAD_cond1 = (merge['DataSource'] == 2) & (merge['ProcessName'].str.split('/', expand=True)[1].astype('Int64') >= 2015)

# תכנית נמצאת במערכת ניהול חגי
CAD_cond2 = merge['ProcessName'].isin(hagi_table['ProcessName'])

# קטן משנת 2015 לפי שנת אליפסה
SRV_cond = (merge['DataSource'] == 3) & (merge['ProcessName'].str.split('/', expand=True)[1].astype('Int64') < 2015)


tabular_cond = merge['DataSource'] == 4 #טבלאי


card_cond = merge['DataSource'] == 5 # כרטסת


merge['DataSource'] = np.where(digi_cond, 3, None)
merge['DataSource'] = np.where(tabular_cond, 2, merge['DataSource'])
merge['DataSource'] = np.where(card_cond, 1, merge['DataSource'])
merge['DataSource'] = np.where(SRV_cond, 4, merge['DataSource'])
merge['DataSource'] = np.where(CAD_cond1, 5, merge['DataSource'])
merge['DataSource'] = np.where(CAD_cond2, 5, merge['DataSource'])

merge['DataSource'] = merge['DataSource'].fillna(3)

merge = merge.drop(columns = ['DataSource_Borders', 'DataSource_Tabular', 'DataSource_Arcstorm'])

del hagi_table

In [ ]:
merge['ORACLE_ID'] = merge['ORACLE_ID_Borders'].fillna(merge['ORACLE_ID_Tabular'])
merge = merge.drop(columns = ['ORACLE_ID_Borders', 'ORACLE_ID_Tabular'])

In [ ]:
merge['SurveyorLicenseID'] = merge['SurveyorID'].fillna(merge['SurveyorID_Oracle.InProcess'])\
                                                .fillna(merge['SURVEYOR_Arcstorm'])\
                                                .fillna(merge['SURVEYOR_Hagi'])

merge = merge.drop(columns = ['SurveyorID', 'SurveyorID_Oracle.InProcess', 'SURVEYOR_Arcstorm', 'SURVEYOR_Hagi'])

In [ ]:
merge['geometry'] = merge['geometry_Borders'].fillna(merge['geometry_Tabular'])\
                                             .fillna(merge['geometry_InProcess'])\
                                             .fillna(merge['geometry_Arcstorm'])


merge = merge.set_geometry('geometry', crs = 2039)
merge = merge.drop(columns = ['geometry_Borders', 'geometry_Tabular', 'geometry_InProcess', 'geometry_Arcstorm'])

In [ ]:
merge['MapYear'] = merge['SurveyDate'].dt.year
merge['MapYear'] = merge['MapYear'].fillna(merge['ProcessName'].str.split('/', expand=True)[1])
merge['MapYear'] = merge['MapYear'].astype(int)

geodeticNetDict = {range(0, 1998)     : 1,      # רשת ישראל הישנה
                   range(1999, 2007)  : 2,      # רשת ישראל החדשה
                   range(2008, 99999) : 3}      # רשת ישראל התקפה

merge['GeodeticNetwork'] = merge['MapYear'].apply(lambda x: next((v for k, v in geodeticNetDict.items() if x in k), 0))
merge = merge.drop(columns = ['SurveyDate', 'MapYear'])

del geodeticNetDict

In [ ]:
CadBlocks = Blocks[Blocks['IsTax'] == 0].drop(columns = 'IsTax').rename(columns = {'GlobalID' : 'BlockUniqueID', 'Name' : 'BlockName'})

TaxBlocks = Blocks[Blocks['IsTax'] == 1].drop(columns = 'IsTax')\
                                        .drop_duplicates('Name', keep = False)\
                                        .rename(columns = {'GlobalID' : 'TaxBlockUniqueID', 'Name' : 'BlockName'})


merge = merge.merge(CadBlocks, on = 'BlockName', how = 'left')\
             .merge(TaxBlocks, on = 'BlockName', how = 'left')\
             .drop(columns = 'BlockName')

merge['BlockUniqueID'] = merge['BlockUniqueID'].fillna(merge['TaxBlockUniqueID'])

merge.drop(columns = 'TaxBlockUniqueID', inplace = True)

del [Blocks, CadBlocks, TaxBlocks]

In [ ]:
int_cols = ['Status', 'DataSource', 'ORACLE_ID', 'SurveyorLicenseID', 'GeodeticNetwork']
merge[int_cols] = merge[int_cols].astype('Int64')
del int_cols

<h1 style="text-align:right; font-size:125%">
:קיבוץ טבלה
<h1 style="text-align:right; font-size:100%">
הקיבוץ יאחד כל תצ"ר לפי מספר האליפסה וישמור לכל תצ"ר את הערך הנכון בשדות

<h1 style="text-align:right; font-size:100%">
ניקוי ערכי שם תב"ע ארוכים מדי לערך ריק

In [ ]:
merge['PlanName_len'] = merge['PlanName'].str.len()
merge.loc[merge['PlanName_len'] >= 35, ['PlanName']] = None
merge.drop(columns = 'PlanName_len', inplace = True)

In [ ]:
merge = merge[merge['ProcessName'] != '0']
merge = merge[merge['ProcessName'].notna()]

In [ ]:
merge = merge.dissolve('ProcessName', as_index = False, aggfunc = 'first')

In [ ]:
merge['ProcessType'] = 1

<h1 style="text-align:right; font-size:150%; color:#1c6ce6;"> 
יצירת שכבה חדשה בהתאם לאפיון ✨

In [ ]:
print('   Defining new Layer')

<h1 style="text-align:right; font-size:100%">
איכלוס שדות

<h1 style="text-align:right; font-size:100%">
הסרת כפילויות

<h1 style="text-align:right; font-size:150%; color:#1c6ce6;"> 
שמירת תוצאות 💾
<h1 style="text-align:right; font-size:100%;">

<h1 style="text-align:right; font-size:100%;">     
פיצול התוצאות לפי סוגי גיאומטריה שונים (פעולה טכנית כי לא ניתן לכתוב קובץ סוגי גיאומטריה שונים באותה שכבה)    
<h1 style="text-align:right; font-size:100%;"> 
שמירת שכבה גיאוגרפית 

In [ ]:
print('   Exporting results')

In [ ]:
Polygons      = merge[(~merge['geometry'].isna()) & (merge['geometry'].geom_type == 'Polygon')]
MultiPolygons = merge[(~merge['geometry'].isna()) & (merge['geometry'].geom_type == 'MultiPolygon')]
EmptyGeometry = merge[merge['geometry'].isna()]

In [ ]:
Polygons.to_file(CNFG.Results, layer = 'PT1', driver = 'OpenFileGDB', engine = 'pyogrio', mode = 'w')
MultiPolygons.to_file(CNFG.Results, layer = 'PT1', driver = 'OpenFileGDB', engine = 'pyogrio', mode = 'a') 
EmptyGeometry.to_file(CNFG.Results, layer = 'PT1', driver = 'OpenFileGDB', engine = 'pyogrio', mode = 'a')